# Homework Two:  Context-Free Grammars and Shift-Reduce Parsing

Please provide answers to these problems by filling in this notebook and submitting it via websubmit by Tuesday 2/6 by midnight.  For the second problem, you must also submit an electronic version of your drawings, either using drawing software or by scanning or photographing hand drawings. Make sure in the latter case that the result is legible; if we can not understand your drawing, it will receive no credit. 

You may submit by Wednesday midnight for a 20% penalty; after that no credit will be given. 

I will drop the lowest homework/lab combination at the end of the term.

For coding problems any <i> reasonable </i> solution, in which you understand the spirit of the exercise and try to learn this week's material through your efforts, will be graded based solely on the test cases. In general, shortcuts to avoid learning the material will be unreasonable. For example, you may not use any Python libraries or functions that make your task dramatically easier (e.g., you may use split(), replace(...), int(...), and float(...), as mentioned below, and any other normal functions, but may NOT use the Python library for regular expressions).  Good Python style, using list comprehensions, Numpy, creating helper functions in addition to what is required, etc., <i>when appropriate,</i> are quite reasonable! Check with me if you have questions about this. 

<b> Make sure to click Cell -> Run All before submitting notebooks!!</b>

## Problem One (10 pts)

Give context-free grammars for the following languages. 
<ol style="list-style-type: lower-alpha;">
      <li>The language of matching delimiters over the alphabet: <pre>
      {    }    [    ]    (    )        
</pre>   
   
   the following are in the language:  <code>   (())    ({})    {([]())}{}</code> <br>
   the following are not:   <code>   ({)}   {{{}})  </code>  $\epsilon$ (empty string)<br>

</li>      
      <li> The language $\{ a^n b^m a^n |\, n \ge 0 \text{ and } m > 1 \}$
      <li> The language represented by the regular expression $a\, b^\ast\, (\,c\,|\,d\,)$.
      <li> The language over $\{ a,b\}$ of strings with an odd number of a's and any number of b's. 
      <li>The language over alphabet $\{a,b\}$ where the number of $a$'s = the number of $b$'s (any length, including 0). 
</ol>
Hint: For c and d, the languages are regular: simulate a DFA by associating a non-terminal with each state, and then write rules which make "transitions" from one state to another; each rule will have a single non-terminal in the last position on the right-hand side. 

## Solution
<pre>
Your answer here

</pre>

## Problem Two (5 pts)

This is a problem showing how difficult it sometimes is to create non-ambiguous grammars for standard programming language constructs. It is called the "dangling else problem."

(a) Show that the following obvious grammar for conditional statements is ambiguous by showing
two different parse trees for the input <code> if expr then if expr then stmt else stmt</code>.      
<pre>   
     S := if expr then S
     S := if expr then S else S
     S := stmt
</pre>    

(b) Now consider the following, more complicated grammar, which is NOT ambiguous:
<pre>
    S :=  M
    S := U
    M := if expr then M else M
    M := stmt
    U := if expr then S
    U := if expr then M else U
</pre>
(M representes "matched then's" and U representes "unmatched then's."
Show the parse tree for the dangling-else example from (a) using this new grammar. 

## Solution
<pre>
Your answer here

</pre>

## Problem Three (10 pts)

For each of the following either give some proof/argument/intuition as to why it is true, or give a counter-example showing it is not true. If it is true, an example may illustrate the intuition, but is not sufficient by itself. 

<p>Let $G$ be a grammar and $w$ be a string of terminal symbols.

<ol style="list-style-type: lower-alpha;">
      <li> If each rule in $G$ has most one non-terminal on the right-hand side, then any left-most derivation of a string $w$ is also a right-most derivation. 
      <li> $G$ is ambiguous if and only if there are two different right-most derivations of some string $w$. 
      <li> If there are two distinct derivations (of any kind) of $w$ in $G$, then $G$ is ambiguous. 
      <li> If $G$ is not ambiguous, then every derivation of $w$ in $G$ will have the same number of derivation steps (those indicated by => in the derivations). 
      <li> If $G$ is ambiguous, then there are an infinite number of strings $w$ each of which has more than one parse tree.  
</ol>

## Solution
<pre>
Your answer here

</pre>

# Problem Four  Parser (20 pts)

Here is the grammar similar to the one discussed in lecture for the language of arithmetic expressions (we have added a rule for parenthesized expressions):
<pre>
0:   S := E $
1:   E := E + T
2:   E := T
3:   T := T * F
4:   T := F
5:   F := ( E )
6:   F := int

</pre>

In the rest of this problem, you will add code to build a shift-reduce parser for this grammar. The transition table is given, and you have to supply code to complete the parser. 


## Lexer Code: Insert your lexer code in this next cell (or use my posted solution)

In [267]:
# Insert your lexer here or use my solution (posted Friday am)

## Helper Code (do not modify)

In [2]:
# The symbols of the
# terminals are encoded by pairs (category,etc) according to:

ident = 0           # not used in this hw, but leave it in for the future!
integer = 1
floating = 2        # not used in this hw
plus = 3
minus = 4           # not used in this hw
mult = 5
div = 6             # not used in this hw
lparen = 7
rparen = 8
comma = 9           # not used in this assignment
error = 10        

# Non-terminals are encoded as pairs using these codes:

S = 11             # these will be represented as tuples, e.g., (S,), just like terminals
E = 12
T = 13
F = 14

# special token for end of input

end_of_input = 15       # (end_of_input,) will be pretty-printed as ($,)


# pretty-printing lists of tokens

tokenCats = ['ident','integer','floating','plus','minus','mult','div',
             'lparen','rparen','comma','error','S','E','T','F','$']

def tokenToString(t):
    if t == None:
        return str(t)
    elif t[0] < 3:
        return '(' + tokenCats[t[0]] + ',' + str(t[1]) + ')'
    else:
        return '(' + tokenCats[t[0]] + ',)'
        
def tokenListToString(lst):
    res = '[ '
    for t in lst[:-1]:
        res = res + tokenToString(t) + ', '
    res = res + tokenToString(lst[-1]) + ' ]'
    return res

# pretty-printing parser configurations

def pprintParser(parsingStack,inputListOfTokens):
    totalWidth = 80
    smallestGap = 6
    largestStackLength = int(totalWidth*0.6 - smallestGap/2)   # most characters to display
    largestInputLength = totalWidth - largestStackLength - smallestGap
    
    p = '| '
    for symbol in parsingStack:
        p = p + tokenToString(symbol) + ' '
    if len(p) > largestStackLength:
        ind = int(len(p) - largestStackLength + 9)
        p = '| ... ' + p[ind:]
        
    q = ""
    for tok in inputListOfTokens[:-1]:
        q = q + tokenToString(tok) + ' '
    if len(inputListOfTokens) > 0:
        q = q + tokenToString(inputListOfTokens[-1]) 

    if len(q) > largestInputLength:
        ind = int(largestInputLength - 9)
        q = q[:ind] + ' ... ' 
    
    q = q + ' |'
        
    p = p + (' ' * (totalWidth - len(p) - len(q)))
    print(p+q)

# pretty-printing rules

def toStringRule(r):
    s = tokenCats[r[0][0]] + ' := '
    for t in r[1]:
        s = s + tokenCats[t[0]] + ' '
    return s
                          
def toStringRules(lst): 
    s = ""
    for r in lst:
        s = s + toStringRule(r) + '\n'
    return s

## Part (A): Encoding the rules

Rules will be represented by pairs (lhs, rhs) where lhs is a single non-terminal tuple
and rhs is a list of tuples for terminals or non-terminals. For example, the start
rule would be represented as follows:
<pre>
((S,),[(E,),(end_of_input,)])
</pre>
and the last rule would be
<pre>
((F,),[(integer,)])
</pre>
In this last rule, you don't actually need the value of the integer token (only the category of token
is used during parsing), so you can just leave it out. 

In the next cell, encode the list of rules in the order given above. This will be useful 

In [3]:
rules = [      ]                    



print("Grammar:\n\n" + toStringRules(rules))

Grammar:




## Part B: Implement the DFA

<p> Here is a diagram of the DFA which will be used to read the stack and decide 
when to do reduce moves. The error state (numbered -100) is implicit. 

![DFA](http://www.cs.bu.edu/fac/snyder/cs320/Homeworks/HW02Diagram.jpg "DFA")

We have provided the DFA in the next cell in the form of a transition table. You must complete the definition of the action to be taken when the DFA is applied to the parsing stack plus the "lookahead" symbol (next symbol in the input).

In [4]:
# transition table   DO NOT MODIFY UNLESS YOU REALLY KNOW WHAT YOU ARE DOING!

# special actions used by the parser

err = -100
accept = 100

table = { 1: { 1:6, 2:err, 3:err, 4:err, 5:err, 6:err, 7:9, 8:err, 9:err, 10:err, 11:err, 12:2, 13:4, 14:13, 15:err}, 
          2: { 1:err, 2:err, 3:8, 4:err, 5:err, 6:err, 7:err, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:accept}, 
          3: { 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0, 15:0}, 
          4: { 1:-2, 2:-2, 3:-2, 4:-2, 5:5, 6:-2, 7:-2, 8:-2, 9:-2, 10:-2, 11:-2, 12:-2, 13:-2, 14:-2, 15:-2}, 
          5: { 1:6, 2:err, 3:err, 4:err, 5:err, 6:err, 7:9, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:12, 15:err}, 
          6: { 1:-6, 2:-6, 3:-6, 4:-6, 5:-6, 6:-6, 7:-6, 8:-6, 9:-6, 10:-6, 11:-6, 12:-6, 13:-6, 14:-6, 15:-6}, 
          7: { 1:-1, 2:-1, 3:-1, 4:-1, 5:5, 6:-1, 7:-1, 8:-1, 9:-1, 10:-1, 11:-1, 12:-1, 13:-1, 14:-1, 15:-1}, 
          8: { 1:6, 2:err, 3:err, 4:err, 5:err, 6:err, 7:9, 8:err, 9:err, 10:err, 11:err, 12:err, 13:7, 14:13, 15:err}, 
          9: { 1:6, 2:err, 3:err, 4:err, 5:err, 6:err, 7:9, 8:err, 9:err, 10:err, 11:err, 12:10, 13:4, 14:13, 15:err}, 
         10: { 1:err, 2:err, 3:8, 4:err, 5:err, 6:err, 7:err, 8:11, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err}, 
         11: { 1:-5, 2:-5, 3:-5, 4:-5, 5:-5, 6:-5, 7:-5, 8:-5, 9:-5, 10:-5, 11:-5, 12:-5, 13:-5, 14:-5, 15:-5},
         12: { 1:-3, 2:-3, 3:-3, 4:-3, 5:-3, 6:-3, 7:-3, 8:-3, 9:-3, 10:-3, 11:-3, 12:-3, 13:-3, 14:-3, 15:-3},
         13: { 1:-4, 2:-4, 3:-4, 4:-4, 5:-4, 6:-4, 7:-4, 8:-4, 9:-4, 10:-4, 11:-4, 12:-4, 13:-4, 14:-4, 15:-4}}







# this reads a list of tokens (i.e., the parsing stack) and returns the state where you end up

def action(lst,table):
    pass 


# TESTS
# You may not modify the following in any way
# All tests should return True

print(action([(lparen,)],table)==9)
print(action([(E,),(plus,), (F,)],table) == 13)
print(action([(error,),(E,)],table) == err)
print(action([(lparen,),(lparen,),(lparen,),(lparen,)],table)== 9)
print(action([(lparen,),(lparen,),(E,),(rparen,),(plus,)],table) == -5)
print(action([(E,),(end_of_input,)],table) == accept)

False
False
False
False
False
False


## Part C: Implement the driver for the parser

Most of this has been done, just complete where you see pass

In [6]:
# parse takes a list of tokens and determines if it is a 
# well-formed arithmetic expression.

def parse(list_of_input_tokens,rules,table,verbose=False):
    
    # add end_of_input marker to input
    pass
    
    # input stack; use pop(0) to remove front item in list
    input_stack = list_of_input_tokens
    
    # parsing stack; use append to push onto end and pop() to pop from end
    parsing_stack = []
    
    if verbose:
        print('Input Tokens: ' + tokenListToString(input_stack) + '\n')
        pprintParser(parsing_stack,input_stack) 
    
    while( len(input_stack) > 0 ):
        
        # Now we will "lookahead" at the next symbol on the input and ask the automaton what to do
        # a positive number means to shift, and a negative number -n means to reduce by rule n
        
        # complete this next line by calling action on the parsing stack plus the
        # next symbol on the input
        
#        n = 
        if n == accept:   # reduce by start rule, success
            if verbose:
                pprintParser(parsing_stack,input_stack)
            return True
        elif n == err:     #  problem on stack!
            if verbose:
                pprintParser(parsing_stack,input_stack)
            return False 
        elif n > 0:     # shift          
            pass         
        else:         # reduce by rule -n            
            pass
   
        if verbose:
            pprintParser(parsing_stack,input_stack)
            
    return None     # this will never be executed


## Parser Tests 

Last two are incorrect and will return False, rest are correct. 

In [7]:
# Test 1: Correct parse

s = '2'
parse(lexer(s),rules,table,True)

NameError: name 'lexer' is not defined

In [8]:
s = '(2 + 4)'
parse(lexer(s),rules,table,True)

NameError: name 'lexer' is not defined

In [9]:
s = '2+4*8+5'
parse(lexer(s),rules,table,True)

NameError: name 'lexer' is not defined

In [10]:
s = '((2 + (4)) + 4)'
parse(lexer(s),rules,table,True)

NameError: name 'lexer' is not defined

In [11]:
s = '2 * 5 + 2 * 2 + 4'
parse(lexer(s),rules,table,True)

NameError: name 'lexer' is not defined

In [12]:
s = '(2 + )'
parse(lexer(s),rules,table,True)

NameError: name 'lexer' is not defined

In [13]:
s = '(2 + 4.5)'
parse(lexer(s),rules,table,True)

NameError: name 'lexer' is not defined